In [ ]:
from astropy import units
import numpy as np
import pylab as plt
from scipy import signal

import disco

import utils

In [ ]:
hist = disco.ParticleHistory.load('triple_dispersion.h5')
hist

In [ ]:
disp_sim = utils.setup_dispersion_sim()

particle_inv_lat, particle_energy = np.meshgrid(
    disp_sim.particle_invlat_axis,
    disp_sim.particle_energy_axis,
    indexing='ij'
)

In [ ]:
mp = 1.6726e-24
me = 9.1094e-28
energy1eV = 1.6022e-12
cspeed = 2.9979e10
kb = 1.3807e-16
re = 6378.14*1e5
echarge = 4.8032e-10
species = 'ion'
temp_fac = 7


In [ ]:
density = hist.extra_fields['n']
vx = hist.extra_fields['ux']*1e5
vy = hist.extra_fields['uy']*1e5
vz = hist.extra_fields['uz']*1e5
pressure = hist.extra_fields['p']*1e-9*10

jx = hist.extra_fields['jx']*1e-6*3e9
jy = hist.extra_fields['jy']*1e-6*3e9
jz = hist.extra_fields['jz']*1e-6*3e9

bx = hist.extra_fields['Bx']*1e-9*1e4
by = hist.extra_fields['By']*1e-9*1e4
bz = hist.extra_fields['Bz']*1e-9*1e4

bmag = np.sqrt(bx**2+by**2+bz**2)
bhatx = bx/(bmag+1e-30)
bhaty = by/(bmag+1e-30)
bhatz = bz/(bmag+1e-30)

jpar = jx*bhatx+jy*bhaty+jz*bhatz
jperp_x = jx-jpar*bhatx
jperp_y = jy-jpar*bhaty
jperp_z = jz-jpar*bhatz
jperp = np.sqrt(jperp_x**2+jperp_y**2+jperp_z**2)

In [ ]:

import tqdm

efluxes = np.nan * np.zeros(particle_energy.size)
nmaxes = np.nan * np.zeros(particle_energy.size)
fdists = np.nan * np.zeros(particle_energy.size)

for i in tqdm.tqdm(range(particle_energy.size)):
    energy = particle_energy.flatten()[i].to_value(units.eV)
    end_idx = np.searchsorted(hist.stopped[:, i], 1)
    if end_idx == particle_energy.shape[0]:
        continue
    
    nsrc = density[:end_idx, i]
    psrc = pressure[:end_idx, i]
    bsrc = bmag[:end_idx, i]
    jsrc = jperp[:end_idx, i]

    nsrc = nsrc[::-1]
    psrc = psrc[::-1]
    bsrc = bsrc[::-1]
    jsrc = jsrc[::-1]

    temp_ion = psrc/nsrc/energy1eV*temp_fac/(1.+temp_fac)
    vth_ion = np.sqrt(psrc/nsrc/mp*temp_fac/(1.+temp_fac))
    valf_ion = bsrc/np.sqrt(4.*np.pi*nsrc*mp)

    temp_ele = psrc/nsrc/energy1eV/(1.+temp_fac)
    vth_ele = np.sqrt(psrc/nsrc/me/(1.+temp_fac))
    valf_ele = bsrc/np.sqrt(4.*np.pi/nsrc*me)

    if species == 'ele':
        temp = temp_ele
        vth = vth_ele
        valf = valf_ele
        mass = me
    elif species == 'ion':
        temp = temp_ion
        vth = vth_ion
        valf = valf_ion
        mass = mp
    
    """ impulsive BGK model """
    idx_peaks = signal.find_peaks(jsrc)[0]
    eps = jsrc/nsrc/echarge*vth_ion/valf_ion**2
    idx_eps = np.where(eps >= 0.1)[0]
    idx = np.intersect1d(idx_peaks, idx_eps)
    nmax = len(idx)
    
    if nmax == 0:
        idx = 0
        fdist = nsrc[idx]/vth[idx]**3./(2.*np.pi)**(3./2.)*np.exp(-energy/temp[idx])
        eflux = 2.*(energy*energy1eV)**2/mass**2*fdist
    else:
        fdist = nsrc[0]/vth[0]**3./(2.*np.pi)**(3./2.)*np.exp(-energy/temp[0])*np.exp(-nmax)
        eflux = 2.*(energy*energy1eV)**2/mass**2*fdist
        for ii in range(nmax):
            fd = nsrc[idx[ii]]/vth[idx[ii]]**3./(2.*np.pi)**(3./2.)*np.exp(-energy/temp[idx[ii]])
            ef = 2.*(energy*energy1eV)**2/mass**2*fd
            fdist = fdist + fd*np.exp(-(nmax-1-ii))
            eflux = eflux + ef

    efluxes[i] = eflux
    nmaxes[i] = nmax
    fdists[i] = fdist

In [ ]:
plt.pcolor(np.degrees(particle_inv_lat), particle_energy, np.log10(nmaxes).reshape(particle_inv_lat.shape))
plt.yscale('log')
plt.colorbar().set_label('Log10(nmax)')
plt.xlabel('Invariant Magnetic Latitude (degrees)')
plt.ylabel('Energy (eV)')

In [ ]:
plt.pcolor(np.degrees(particle_inv_lat), particle_energy, efluxes.reshape(particle_inv_lat.shape), vmax=1e9)
plt.yscale('log')
plt.colorbar()
plt.title('Ion Flux')
plt.xlabel('Invariant Magnetic Latitude (degrees)')
plt.ylabel('Energy (eV)')